<a href="https://colab.research.google.com/github/Hamideh-ghanadian/ChatGPT_for_Suicide_Risk_Assessment_on_Social_Media/blob/main/DistilBERT_umd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 22.2 MB/s eta 0:00:

In [ ]:
import pandas as pd
from datasets import load_dataset

In [ ]:
# data_path = '/content/drive/MyDrive/PhD/Suicide/Datasets/dataset_1.csv'
# data = pd.read_csv('/content/drive/MyDrive/PhD/Suicide/Datasets/dataset_1.csv', encoding='UTF-8')
# data = pd.read_csv('/content/drive/MyDrive/PhD/Colab_Notebooks/Model/data_and_results/500_Reddit_users_posts_labels/500_Reddit_users_posts_labels.csv', encoding='UTF-8')
data = pd.read_csv('/content/drive/MyDrive/PhD/Colab_Notebooks/UMD_dataset_processing /data/concat_labeled-data.csv', encoding='UTF-8')
data.head(10)

,user_id,label,post_body
0,152,a,Need someone to talk to nan
1,706,a,_URL_ Please head over to this link.He needs h...
2,1288,a,*_PERSON_ do not have to be good. You do not h...
3,1500,a,So my friend (m 17) has been diagnosed clinica...
4,1690,a,"_PERSON_ this in another sub, felt I needed to..."
5,2071,a,It's a lie.I have been feeling like this for t...
6,2103,a,Copy+paste from Op. This happened starting Fri...
7,2435,a,"Hi everyone, it's my first time posting here. ..."
8,3344,a,Hay guys.One of my close friends whom I met on...
9,4192,a,From r/trees. Wanted to offer my ears to anybo...


In [ ]:
# test = pd.read_csv('/content/drive/MyDrive/PhD/Suicide/Datasets/twitter_unlabeled.csv', encoding='UTF-8')
# test.head(10)

# clean=test.dropna()
# test= clean.reset_index(drop = True)
# Labels = list(test['Label'].unique())

# test_dataset = []

# for index,row in test.iterrows():
#   item ={'text':row['text'][2:-2]}
#   test_dataset.append(item)
# test_dataset
# test_data= pd.DataFrame(test_dataset)
# test_data.head(10)



In [ ]:
clean=data.dropna()
data= clean.reset_index(drop = True)
data.head(10)


,user_id,label,post_body
0,152,a,Need someone to talk to nan
1,706,a,_URL_ Please head over to this link.He needs h...
2,1288,a,*_PERSON_ do not have to be good. You do not h...
3,1500,a,So my friend (m 17) has been diagnosed clinica...
4,1690,a,"_PERSON_ this in another sub, felt I needed to..."
5,2071,a,It's a lie.I have been feeling like this for t...
6,2103,a,Copy+paste from Op. This happened starting Fri...
7,2435,a,"Hi everyone, it's my first time posting here. ..."
8,3344,a,Hay guys.One of my close friends whom I met on...
9,4192,a,From r/trees. Wanted to offer my ears to anybo...


In [ ]:
labels = list(data['label'].unique())
labels

['a', 'b', 'c', 'd']

In [ ]:
# #numeric_labels =[labels.index(label) for label in labels]
# label_map =  dict(zip(labels, [1,0]))  #Isar:usually suicide is the positive class
# label_map
numeric_labels =[labels.index(label) for label in labels]
label_map =  dict(zip(labels, numeric_labels))
label_map

{'a': 0, 'b': 1, 'c': 2, 'd': 3}

In [ ]:
dataset = []
for index,row in data.iterrows():
  item ={'label': label_map[row['label']],
         'Post':row['post_body']}
  dataset.append(item)

In [ ]:
dataset_df = pd.DataFrame(dataset)

In [ ]:
dataset_df

,label,Post
0,0,Need someone to talk to nan
1,0,_URL_ Please head over to this link.He needs h...
2,0,*_PERSON_ do not have to be good. You do not h...
3,0,So my friend (m 17) has been diagnosed clinica...
4,0,"_PERSON_ this in another sub, felt I needed to..."
...,...,...
860,3,My story isn't that special. I lived a decent ...
861,2,&lt;After writing this I thought I should give...
862,3,"Hi. I'm 16, I'm a girl, and I'm ready to die. ..."
863,2,Just browse my post history. Go to my very fir...


In [ ]:
##temporary ####remove this
# train_data = pd.DataFrame()
# train_data['text'] = list(dataset_df['selftext'].values)
# train_data['label'] = list(dataset_df['is_suicide'].values)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(list(dataset_df['Post'].values), list(dataset_df['label'].values), test_size=0.20, random_state=42,shuffle=True)

In [ ]:
len(y_test)

173

In [ ]:
data_path = '/content/drive/MyDrive/PhD/Colab_Notebooks/UMD_dataset_processing /data'

train_data = pd.DataFrame()
train_data['Post'] = X_train
train_data['label'] = y_train
# train_data = train_data.to_csv(data_path +'train1.csv')

test_data = pd.DataFrame()
# test_data['Post'] = X_test
# test_data['label'] = y_test
test_data['Post'] = list(dataset_df['Post'].values)
test_data['label'] = list(dataset_df['label'].values)
# test_data = test_data.to_csv(data_path +'test1.csv') 

In [ ]:
from transformers import AutoTokenizer,AlbertConfig, AlbertModel
import torch
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4) 

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier

In [ ]:
train_encodings = tokenizer(train_data['Post'].values.tolist(), truncation=True, padding=True,return_tensors="pt", max_length=512)
train_labels =train_data['label'].values.tolist()  #placeholders, not real labels

train_dataset = MyDataset(train_encodings, train_labels)

In [ ]:
train_dataset

In [ ]:
test_encodings = tokenizer(test_data['Post'].values.tolist(), truncation=True, padding=True,return_tensors="pt", max_length=512)
test_labels =test_data['label'].values.tolist()  #placeholders, not real labels

test_dataset = MyDataset(test_encodings, test_labels)

In [ ]:
test_dataset

In [ ]:
# # test_encodings = tokenizer(test_data['text'].values.tolist(), truncation=True, padding=True,return_tensors="pt", max_length=512)
# # test_labels =test_data['class'].values.tolist()  #placeholders, not real labels

# # test_dataset = MyDataset(test_encodings, test_labels)


# #For labeling
# test_encodings = tokenizer(test_data['text'].values.tolist(), truncation=True, padding=True,return_tensors="pt", max_length=512)
# test_labels =test_data['text'].values.tolist()  #placeholders, not real labels

# test_encodings
# # test = torch.utils.data.TensorDataset(torch.tensor(test_encodings['input_ids']),torch.tensor(test_encodings['token_type_ids']),torch.tensor(test_encodings['attention_mask']))

# # class MyDataset1(torch.utils.data.Dataset):
# #     def __init__(self, encodings):
# #         self.encodings = encodings
        

# #     def __getitem__(self, idx):
# #         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
# #         return item

# #     def __len__(self):
# #             return len(self.encodings)

# test = MyDataset(test_encodings,test_labels)  
# test




In [ ]:
import numpy as np
from datasets import load_metric
!pip install sklearn
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(eval_pred):
    metric = load_metric("accuracy")
    metrics = ['recall', 'precision', 'f1']

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Compute accuracy
    acc = metric.compute(predictions=predictions, references=labels)

    # Compute precision, recall, and F1 score for each class
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')

    # Create a dictionary to store the results
    metric_res = {'accuracy': acc['accuracy'], 'precision': precision, 'recall': recall, 'f1': f1}

    return metric_res


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2955 sha256=e1d5f0084cc5fd5a0cb3ca8ad80fb4f69e33a7b9a0c512bbd7ac23b6ba462669
  Stored in directory: /root/.cache/pip/wheels/f8/e0/3d/9d0c2020c44a519b9f02ab4fa6d2a4a996c98d79ab2f569fa1
Successfully built sklearn


In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/PhD/Colab_Notebooks/Model/data_and_results/distilbert_umd/out",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=7,
    weight_decay=0.01,
    evaluation_strategy = "epoch", #To calculate metrics per epoch
    logging_strategy="epoch", #Extra: to log training data stats for loss 
)
import transformers
transformers.logging.set_verbosity_info()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)



PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()
# trainer.train(resume_from_checkpoint = True)

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 692
  Num Epochs = 7
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1211
  Number of trainable parameters = 66956548
<ipython-input-13-26dfdb2d573e>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.765500,0.483693,0.839595,0.843771,0.839595,0.829298
2,0.531600,0.268105,0.904624,0.907773,0.904624,0.896409
3,0.332600,0.156883,0.947977,0.949539,0.947977,0.946400
4,0.175100,0.075948,0.976879,0.976884,0.976879,0.976768
5,0.093400,0.043406,0.988439,0.988616,0.988439,0.988450
6,0.052700,0.030678,0.992775,0.992802,0.992775,0.992778
7,0.035600,0.025407,0.992775,0.992777,0.992775,0.992772


***** Running Evaluation *****
  Num examples = 692
  Batch size = 4
<ipython-input-13-26dfdb2d573e>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
***** Running Evaluation *****
  Num examples = 692
  Batch size = 4
<ipython-input-13-26dfdb2d573e>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Saving model checkpoint to /content/drive/MyDrive/PhD/Colab_Notebooks/Model/data_and_results/distilbert_umd/out/checkpoint-500
Configuration saved in /content/drive/MyDrive/PhD/Colab_Notebooks/Model/data_and_results/distilbert_umd/out/checkpoin

TrainOutput(global_step=1211, training_loss=0.28376412411350344, metrics={'train_runtime': 105.9232, 'train_samples_per_second': 45.731, 'train_steps_per_second': 11.433, 'total_flos': 641694965710848.0, 'train_loss': 0.28376412411350344, 'epoch': 7.0})

In [ ]:
model_dir = '/content/drive/MyDrive/PhD/Colab_Notebooks/Model/data_and_results/distilbert_umd'
trainer.save_model(model_dir + 'trained_model')

Saving model checkpoint to /content/drive/MyDrive/PhD/Colab_Notebooks/Model/data_and_results/distilbert_umdtrained_model
Configuration saved in /content/drive/MyDrive/PhD/Colab_Notebooks/Model/data_and_results/distilbert_umdtrained_model/config.json
Model weights saved in /content/drive/MyDrive/PhD/Colab_Notebooks/Model/data_and_results/distilbert_umdtrained_model/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/PhD/Colab_Notebooks/Model/data_and_results/distilbert_umdtrained_model/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/PhD/Colab_Notebooks/Model/data_and_results/distilbert_umdtrained_model/special_tokens_map.json


In [ ]:
trainer.evaluate(test_dataset)

***** Running Evaluation *****
  Num examples = 865
  Batch size = 4
<ipython-input-13-26dfdb2d573e>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


{'eval_loss': 0.5298583507537842,
 'eval_accuracy': 0.900578034682081,
 'eval_precision': 0.9051451414424987,
 'eval_recall': 0.900578034682081,
 'eval_f1': 0.9014035237496645,
 'eval_runtime': 4.3789,
 'eval_samples_per_second': 197.537,
 'eval_steps_per_second': 49.555,
 'epoch': 7.0}